#### Convert your PDB from Interchange to a GRO with a box large enough to prevent perioidic image interactions

In [ ]:
# Note: The GRO output directly from Interchange may be good enough to skip this step
$ gmx editconf -f Inter_POPC.pdb -o Inter_POPC.gro -box 5 5 5

### Build an index file with an atom from the headgroup

In [ ]:
# The nitrogen from POPC is used here since it is the furthest core atom
# For more complicated structures, you can specificy multiple atoms and link them (reference 'specific_pull.mdp' in this repo)

$ gmx make_ndx -f Inter_POPC.gro -o N1x.ndx

# Input for this index (#10 is the nitrogen in the PDB)
$ a 10
$ name 3 N1x

### Run the pulling simulation

In [ ]:
$ gmx grompp -f pull_nvt.mdp -c Inter_POPC.gro -p Inter_POPC.top -n N1x.ndx -o pull.tpr -maxwarn 1
$ gmx mdrun -deffnm pull

### Visualize the trajectory in VMD and dump a frame where the lipid is thin/ideal as a PDB

In [ ]:
$ gmx trjconv -f pull.xtc -s pull.tpr -o POPC.pdb -dump 73
# Where '73' is the best frame

# Note: The GRO output directly from the md may be good enough to skip this step. If so, use this
$ gmx editconf -f pull.gro -o POPC.pdb

### Run packmol with the dumped PDB

In [ ]:
$ packmol < bilayer-comment.inp

# If packmol is not installed, create a Conda environment
$ conda create -n bilayer
$ conda activate bilayer
$ conda install -c conda-forge packmol

### Convert PDB to GRO using the same dimensions as your system

In [ ]:
$ gmx editconf -f bilayer.pdb -o bilayer.gro -box 7.5 7.5 7.5

### Energy minimization (gmx start here)

In [ ]:
$ gmx grompp -f min.mdp -p topol.top -c bilayer.gro -o min.tpr
$ gmx mdrun -deffnm min

### NVT equilibration

In [ ]:
$ gmx grompp -f nvt.mdp -p topol.top -c min.gro -o nvt.tpr
$ gmx_mpi mdrun -deffnm nvt

### NPT equilibration

In [ ]:
$ gmx grompp -f npt.mdp -p topol.top -c nvt.gro -o npt.tpr -maxwarn 1
$ gmx mdrun -deffnm npt

### MD Production

In [ ]:
$ gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr
$ gmx mdrun -deffnm md

# Checkpoint run for completion
$ gmx mdrun -deffnm md -cpi md.cpt